In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import datetime as dt
from itertools import product

from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm

from weather_resource import vector_magnitude

In [3]:
# Define data clean-up functions.
def reset_direction(dataframe):
    dataframe["dir"] = dataframe["dir"] % 360
    return dataframe


def pandas_to_xarray(dataframe):

    # Simplify variable name.
    df = dataframe

    # Define columns of interest.
    DOFs = [
        "RAOSurgeAmp",
        "RAOSurgePhase",
        "RAOSwayAmp",
        "RAOSwayPhase",
        "RAOHeaveAmp",
        "RAOHeavePhase",
        "RAORollAmp",
        "RAORollPhase",
        "RAOPitchAmp",
        "RAOPitchPhase",
        "RAOYawAmp",
        "RAOYawPhase",
    ]

    # Separate dataframe by degree of freedom.
    dfs = [df[DOF] for DOF in DOFs]

    # Create xarray.DataArray
    da = xr.DataArray(
        data=dfs,
        dims=["DOF", "freq", "dir"],
        coords={
            "DOF": DOFs,
            "freq": dfs[0].index.values,
            "dir": dfs[0].columns.values
        }
    )

    return da

In [4]:
# Apply clean-up and convert to xarray.DataArray.
df = (
    pd.read_csv("./data/preprocessed/MPI_Adventure_RAO.csv")
    .pipe(reset_direction)
    .set_index(["RAOPeriodOrFreq", "dir"])
    .unstack()
    .pipe(pandas_to_xarray)
)

In [5]:
# Import wave data.
waves = pd.read_csv(
    "./data/raw/HKZ_3.970372E_52.014651N.csv",
    skiprows=[0],
    parse_dates={"datetime": ["YYYY", "M", "D", "HH", "MM", "SS"]},
    date_parser=lambda x: dt.datetime.strptime(x, "%Y %m %d %H %M %S")
).set_index("datetime").loc["2019-01-01": "2019-01-04"]

In [6]:
# Create a dataset.
ds = xr.Dataset(
    data_vars=dict(
        launch_rao=(["DOF", "freq", "dir"], df),
        wave_data=(["time", "wave_parameter"], waves)
    ),
    coords=dict(
        DOF=df["DOF"],
        freq=df["freq"],
        dir=df["dir"],
        time=waves.index.values,
        wave_parameter=waves.columns
    )
)

ds

<xarray.Dataset>
Dimensions:         (DOF: 12, dir: 36, freq: 100, time: 96, wave_parameter: 21)
Coordinates:
  * DOF             (DOF) <U13 'RAOSurgeAmp' 'RAOSurgePhase' ... 'RAOYawPhase'
  * freq            (freq) float64 2.374 2.387 2.4 2.413 ... 31.58 42.03 62.83
  * dir             (dir) float64 0.0 10.0 20.0 30.0 ... 320.0 330.0 340.0 350.0
  * time            (time) datetime64[ns] 2019-01-01 ... 2019-01-04T23:00:00
  * wave_parameter  (wave_parameter) object 'Hm0' 'PWD' 'Tp' ... 'MWDS' 'DSDS'
Data variables:
    launch_rao      (DOF, freq, dir) float64 0.002 0.001 0.0 ... 0.422 0.451
    wave_data       (time, wave_parameter) float64 1.343 281.0 ... 340.2 39.94

# Unimodal sea state

In [7]:
# Read from the dataset the Response Amplitude Operator. 
# Then computes the most efficient/workable wave heading. Note:
# - Instead of vector magnitude -> sum is much faster.
# - Coarse grid of possible angles allows for less groups, hence, faster.

# Define a parallelise function.
def apply_parralelise(groups, func, column="launch_rao"):
    
    series = [group[column] for name, group in tqdm(groups)]
    results = []
    
    with Pool(processes=cpu_count()) as p:
        with tqdm(total=len(series)) as pbar:
            for i, res in enumerate(p.imap(func, series)):
                pbar.update()
                results.append(res)

    results = pd.DataFrame(
        index=pd.MultiIndex.from_tuples(groups.groups.keys()),
        data=dict(values=results)
    )

    return results

# Lets compute response motions.
res = ds["launch_rao"].sel(
    indexers=dict(
        freq=ds["wave_data"].sel(dict(wave_parameter="Tp")),
        dir=np.arange(0, 360, 45)  # coarse grid speeds up computation.
        # dir=ds["wave_data"].sel(dict(wave_parameter="MWD"))
    ),
    method="nearest"
)

# Find the index with the smallest response amplitudes.
index = (res.loc[["RAOSurgeAmp", "RAOSwayAmp", "RAOHeaveAmp"]]
    .to_dataframe()
    .reset_index()
    .groupby(["time", "dir"])
    .pipe(lambda g: apply_parralelise(groups=g, func=vector_magnitude))
    .unstack(level=0)
    .idxmin()
    .values
)

  0%|          | 0/768 [00:00<?, ?it/s]

  0%|          | 0/768 [00:00<?, ?it/s]

In [68]:
# Find practical responses.
x_points = xr.DataArray(ds["wave_data"].sel(dict(wave_parameter="Tp")))
y_points = xr.DataArray(index, dims="time")
# motions = ds["launch_rao"].interp(freq=x_points, dir=y_points) * ds["wave_data"].sel(dict(wave_parameter="Hm0"))

In [207]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [347]:
# Find the data of interest.
frequency = xr.DataArray(ds["wave_data"].sel(dict(wave_parameter="Tp")))
angle = xr.DataArray(ds["wave_data"].sel(dict(wave_parameter="MWD")))
raos = ds["launch_rao"].interp(dict(freq=frequency, dir=angle))
heave_motion = raos.loc["RAOHeaveAmp"] * ds["wave_data"].sel(dict(wave_parameter="Hm0"))

allowable_sea_state = pd.read_excel("./data/raw/Pivot_max_Hs_Splash_zone_current_0_DAF_1.2_Angles_1.6874.xlsx", index_col=[0])
allowable_sea_state.index.names = ["dir"]
allowable_sea_state.columns.names = ["freq"]
allowable_sea_state = xr.DataArray(data=allowable_sea_state)

hs_lim = allowable_sea_state.interp(dict(freq=frequency, dir=angle)).fillna(0)

# Create figure.
fig = make_subplots(rows=4, cols=1, row_heights=[0.4, .1, 0.4, .1])

# Add significant wave height.
fig.add_trace(
    go.Scatter(
        x=waves.index,
        y=waves["Hm0"],
        marker=dict(color="blue"),
        showlegend=False
    ),
    row=1, col=1
)

# Add heave line.
fig.add_trace(
    go.Scatter(
        x=waves.index,
        y=heave_motion,
        marker=dict(color="blue"),
        showlegend=False
    ),
    row=3, col=1
)

fig.add_hline(0.2, line=dict(color="red", dash="dash", width=1), row=3, col=1)

fig.add_trace(
    go.Scatter(
        x=waves.index,
        y=hs_lim,
        line=dict(color="red", dash="dash"),
        showlegend=False,
    ),
    row=1, col=1
)

fig.add_annotation(
    text="Threshold level",
    showarrow=False,
    y=0.3,
    row=1, col=1
)

for i in range(len(hswindows)):
    if hswindows.iloc[i]["violated"]:
        color="red"
    else:
        color="green"

    fig.add_trace(
        go.Scatter(
            x=[hswindows.iloc[i]["start"], hswindows.iloc[i]["stop"]],
            y=[4, 4],
            line=dict(width=5, color=color),
            marker=dict(size=8, symbol="line-ns", line=dict(width=1)),
            showlegend=False
            
        ),
        row=2, col=1
    )

for i in range(len(heavewindows)):
    if heavewindows.iloc[i]["violated"]:
        color="red"
    else:
        color="green"

    fig.add_trace(
        go.Scatter(
            x=[heavewindows.iloc[i]["start"], heavewindows.iloc[i]["stop"]],
            y=[.5, .5],
            line=dict(width=5, color=color),
            marker=dict(size=8, symbol="line-ns", line=dict(width=1)),
            showlegend=False
            
        ),
        row=4, col=1
    )

fig.update_xaxes(row=1, col=1, range=[waves.index[0], waves.index[-1]])
fig.update_yaxes(row=1, col=1, title="Sign. wave height [m]")

fig.update_xaxes(row=2, col=1, range=[waves.index[0], waves.index[-1]])
fig.update_yaxes(row=2, col=1, showgrid=False, zeroline=False, showticklabels=False)
fig.update_xaxes(row=2, col=1, showgrid=False, zeroline=False, showticklabels=False)

fig.update_xaxes(row=3, col=1, range=[waves.index[0], waves.index[-1]])
fig.update_yaxes(row=3, col=1, title="Heave amplitude [m]")

fig.update_xaxes(row=4, col=1, range=[waves.index[0], waves.index[-1]])
fig.update_yaxes(row=4, col=1, showgrid=False, zeroline=False, showticklabels=False)
fig.update_xaxes(row=4, col=1, showgrid=False, zeroline=False, showticklabels=False)


fig.update_layout(
    height=600,
    title="Transformation from characteristic parameter to weather windows",
    margin=dict(
        t=50,
        b=10,
        r=20
    )
)



In [262]:
waves["hs_violated"] = waves["Hm0"] > hs_lim
waves["heave_violated"] = heave_motion > 0.2

In [263]:
waves["hs_block"] = (waves["hs_violated"] != waves["hs_violated"].shift()).cumsum()
waves["heave_block"] = (waves["heave_violated"] != waves["heave_violated"].shift()).cumsum()

In [264]:
hswindows = pd.DataFrame(dict(
    start = waves.reset_index().groupby("hs_block")["datetime"].first(),
    stop = waves.reset_index().groupby("hs_block")["datetime"].first().shift(-1),
    violated = waves.reset_index().groupby("hs_block")["hs_violated"].first()
))

heavewindows = pd.DataFrame(dict(
    start = waves.reset_index().groupby("heave_block")["datetime"].first(),
    stop = waves.reset_index().groupby("heave_block")["datetime"].first().shift(-1),
    violated = waves.reset_index().groupby("heave_block")["heave_violated"].first()
))


In [265]:
hswindows.loc[len(hswindows), "stop"] = waves.reset_index().iloc[-1]["datetime"]
heavewindows.loc[len(heavewindows), "stop"] = waves.reset_index().iloc[-1]["datetime"]

In [267]:
hswindows

,start,stop,violated
hs_block,,,
1,2019-01-01 00:00:00,2019-01-01 02:00:00,False
2,2019-01-01 02:00:00,2019-01-04 02:00:00,True
3,2019-01-04 02:00:00,2019-01-04 17:00:00,False
4,2019-01-04 17:00:00,2019-01-04 20:00:00,True
5,2019-01-04 20:00:00,2019-01-04 23:00:00,False
6,2019-01-04 23:00:00,2019-01-04 23:00:00,True
